In [22]:
import torch
from torch.autograd import Function, gradcheck

class CameraProject(Function):
    @staticmethod
    def forward(ctx, x, y, z, fx, fy, cx, cy):
        u = fx * x / z + cx
        v = fy * y / z + cy
        ctx.save_for_backward(x, y, z, fx, fy)
        return u, v

    @staticmethod
    def backward(ctx, grad_u, grad_v):
        x, y, z, fx, fy = ctx.saved_tensors
        grad_x = grad_u * fx / z
        grad_y = grad_v * fy / z
        grad_z = -grad_u * fx * x / z**2 - grad_v * fy * y / z**2
        return grad_x, grad_y, grad_z, None, None, None, None
    
x = torch.tensor(10.0, dtype=torch.float64, requires_grad=True)
y = torch.tensor(-5.0, dtype=torch.float64, requires_grad=True)
z = torch.tensor(10.0, dtype=torch.float64, requires_grad=True)
fx = torch.tensor(1300.0, dtype=torch.float64, requires_grad=False)
fy = torch.tensor(1200.0, dtype=torch.float64, requires_grad=False)
cx = torch.tensor(320.0, dtype=torch.float64, requires_grad=False)
cy = torch.tensor(240.0, dtype=torch.float64, requires_grad=False)

# u, v = CameraProject.apply(x, y, z, fx, fy, cx, cy)

test = gradcheck(CameraProject.apply, (x, y, z, fx, fy, cx, cy))
print(test)


True


In [89]:
import torch
from torch.autograd import Function, gradcheck

class MatrixMultiplication(Function):
    @staticmethod
    def forward(ctx, A, B):
        C = A @ B
        ctx.save_for_backward(A, B)
        return C

    @staticmethod
    def backward(ctx, grad_C):
        A, B, = ctx.saved_tensors
        grad_A = grad_C @ B.T
        grad_B = A.T @ grad_C
        return grad_A, grad_B

    
R = torch.rand(3, 3, dtype=torch.float64, requires_grad=True)
S = torch.rand(3, 3, dtype=torch.float64, requires_grad=True)

test = gradcheck(MatrixMultiplication.apply, (R, S))
print(test)

True


In [88]:
import torch
from torch.autograd import Function, gradcheck

class RSSR(Function):
    @staticmethod
    def forward(ctx, R, S):
        RS = R @ S
        RSSR = RS @ RS.T
        ctx.save_for_backward(R, S)
        return RSSR
    
    @staticmethod
    def backward(ctx, grad_RSSR):
        R, S = ctx.saved_tensors
        RS = R @ S
        grad_RS = grad_RSSR @ RS
        grad_SR = RS.T @ grad_RSSR

        grad_R = grad_RS @ S.T
        grad_S = R.T @ grad_RS

        grad_R_t = S @ grad_SR
        grad_S_t = grad_SR @ R


        return grad_R + grad_R_t.T, grad_S + grad_S_t.T
    

R = torch.rand(3, 3, dtype=torch.float64, requires_grad=True)
S = torch.rand(3, 3, dtype=torch.float64, requires_grad=True)

test = gradcheck(RSSR.apply, (R, S))
print(test)


True


In [87]:
import torch
from torch.autograd import Function, gradcheck

class ComputeSigmaImage(Function):
    @staticmethod
    def forward(ctx, sigma_world, W, J):
        JW = J @ W
        sigma_image = JW @ sigma_world @ JW.T
        ctx.save_for_backward(sigma_world, W, J)
        return sigma_image
    
    @staticmethod
    def backward(ctx, grad_sigma_image):
        sigma_world, W, J = ctx.saved_tensors
        JW = J @ W 

        # using First Quadratic Form 2.3.2 from: https://people.maths.ox.ac.uk/gilesm/files/NA-08-01.pdf
        # for C = B_t @ A @ B 
        # grad_A = B @ grad_C @ B_t
        # grad_B = A @ B @ grad_C_t + A_t @ B @ grad_C
        
        # applying to our variables
        # sigma_image = JW @ sigma_world @ JW.T
        # C = sigma_image
        # A = sigma_world
        # B = JW_t

        grad_sigma_world = JW.T @ grad_sigma_image @ JW
        grad_JW_t = sigma_world @ JW.T @ grad_sigma_image.T + sigma_world.T @ JW.T @ grad_sigma_image

        # compute gradient of JW_t using multiplication rules in 2.2.2 
        grad_W_t =  grad_JW_t @ J
        grad_J_t = W @ grad_JW_t

        grad_W = grad_W_t.T
        grad_J = grad_J_t.T

        return grad_sigma_world, grad_W, grad_J


sigma_world = torch.rand(3, 3, dtype=torch.float64, requires_grad=True)
W = torch.rand(3, 3, dtype=torch.float64, requires_grad=True)
J = torch.rand(2, 3, dtype=torch.float64, requires_grad=True)
test = gradcheck(ComputeSigmaImage.apply, (sigma_world, W, J))
print(test)




True


In [ ]:
import sympy as sp
from sympy import print_latex

def quaternion_to_rotation_Symbolic(q):
    # norm = sp.sqrt(q[0]**2 + x**2 + y**2 + z**2)
    norm = 1
    w = w / norm
    x = x / norm
    y = y / norm
    z = z / norm
    # Compute the rotation matrix
    rotation_matrix = sp.Matrix([[1 - 2*y**2 - 2*z**2, 2*x*y - 2*w*z, 2*x*z + 2*w*y],
                               [2*x*y + 2*w*z, 1 - 2*x**2 - 2*z**2, 2*y*z - 2*w*x],
                               [2*x*z - 2*w*y, 2*y*z + 2*w*x, 1 - 2*x**2 - 2*y**2]])
    
    return rotation_matrix


w, x, y, z = sp.Symbols('w x y z')
q = [w, x, y, z]

rotation_matrix = quaternion_to_rotation_Symbolic(q)
rotation_derivative = sp.diff(rotation_matrix, z)
print_latex(rotation_derivative)



Jacobian of Quaternion to Rotation Matrix without normalization
$$  \frac{\partial q}{\partial w} = \left[\begin{matrix}0 & - 2 z & 2 y\\2 z & 0 & - 2 x\\- 2 y & 2 x & 0\end{matrix}\right] $$

$$  \frac{\partial q}{\partial x} =  \left[\begin{matrix}0 & 2 y & 2 z\\2 y & - 4 x & - 2 w\\2 z & 2 w & - 4 x\end{matrix}\right] $$ 

$$  \frac{\partial q}{\partial y} =  \left[\begin{matrix}- 4 y & 2 x & 2 w\\2 x & 0 & 2 z\\- 2 w & 2 z & - 4 y\end{matrix}\right] $$ 

$$  \frac{\partial q}{\partial z} =  \left[\begin{matrix}- 4 z & - 2 w & 2 x\\2 w & - 4 z & 2 y\\2 x & 2 y & 0\end{matrix}\right] $$ 

In [107]:
import torch
from torch.autograd import Function, gradcheck

class QuaternionToRotation(Function):
    @staticmethod
    def forward(ctx, q):
        rot = [
            1 - 2 * q[:, 2] ** 2 - 2 * q[:, 3] ** 2,
            2 * q[:, 1] * q[:, 2] - 2 * q[:, 0] * q[:, 3],
            2 * q[:, 3] * q[:, 1] + 2 * q[:, 0] * q[:, 2],
            2 * q[:, 1] * q[:, 2] + 2 * q[:, 0] * q[:, 3],
            1 - 2 * q[:, 1] ** 2 - 2 * q[:, 3] ** 2,
            2 * q[:, 2] * q[:, 3] - 2 * q[:, 0] * q[:, 1],
            2 * q[:, 3] * q[:, 1] - 2 * q[:, 0] * q[:, 2],
            2 * q[:, 2] * q[:, 3] + 2 * q[:, 0] * q[:, 1],
            1 - 2 * q[:, 1] ** 2 - 2 * q[:, 2] ** 2,
        ]
        rot = torch.stack(rot, dim=1).reshape(-1, 3, 3)
        ctx.save_for_backward(q)
        return rot

    @staticmethod
    def backward(ctx, grad_rot):
        q = ctx.saved_tensors[0]

        w = q[:, 0]
        x = q[:, 1]
        y = q[:, 2]
        z = q[:, 3]

        grad_qw = -2 * z *grad_rot[:,0 , 1] + 2 * y *grad_rot[:,0, 2] + 2 * z *grad_rot[:,1, 0] - 2 * x *grad_rot[:,1, 2] - 2 * y *grad_rot[:,2, 0] + 2 * x *grad_rot[:,2, 1]
        grad_qx = 2 * y *grad_rot[:,0, 1] + 2 * z *grad_rot[:,0, 2] + 2 * y *grad_rot[:,1, 0] - 4 * x *grad_rot[:,1, 1] - 2 * w *grad_rot[:,1, 2] + 2 * z *grad_rot[:,2, 0] + 2 * w *grad_rot[:,2, 1] - 4 * x *grad_rot[:,2, 2]
        grad_qy = -4 * y *grad_rot[:,0, 0] + 2 * x *grad_rot[:,0, 1] + 2 * w *grad_rot[:,0, 2] + 2 * x *grad_rot[:,1, 0] + 2 * z *grad_rot[:,1, 2] - 2 * w *grad_rot[:,2, 0] + 2 * z *grad_rot[:,2, 1] - 4 * y *grad_rot[:,2, 2]
        grad_qz = -4 * z *grad_rot[:,0, 0] - 2 * w *grad_rot[:,0, 1] + 2 * x *grad_rot[:,0, 2] + 2 * w *grad_rot[:,1, 0] - 4 * z *grad_rot[:,1, 1] + 2 * y *grad_rot[:,1, 2] + 2 * x *grad_rot[:,2, 0] + 2 * y *grad_rot[:,2, 1]
        grad_q = torch.stack([grad_qw, grad_qx, grad_qy, grad_qz], dim=1)

        return grad_q
        

q = torch.rand(10, 4, dtype=torch.float64, requires_grad=True)
norm_q = torch.norm(q, dim=1, keepdim=True)
q = q / norm_q

test = gradcheck(QuaternionToRotation.apply, (q))
print(test)

True


In [ ]:
import sympy as sp
from sympy import print_latex

w, x, y, z = sp.symbols('w x y z')
q = sp.Matrix([w, x, y, z])
norm = sp.sqrt(w**2 + x**2 + y**2 + z**2)

q_norm = q / norm

dw = sp.diff(q_norm, w)
dx = sp.diff(q_norm, x)
dy = sp.diff(q_norm, y)
dz = sp.diff(q_norm, z)

print_latex(dw)
print_latex(dx)
print_latex(dy)
print_latex(dz)

Partial Derivatives of Quaternion Normalization

$$ \frac{\partial q}{\partial w} = \left[\begin{matrix}- \frac{w^{2}}{\left(w^{2} + x^{2} + y^{2} + z^{2}\right)^{\frac{3}{2}}} + \frac{1}{\sqrt{w^{2} + x^{2} + y^{2} + z^{2}}}\\- \frac{w x}{\left(w^{2} + x^{2} + y^{2} + z^{2}\right)^{\frac{3}{2}}}\\- \frac{w y}{\left(w^{2} + x^{2} + y^{2} + z^{2}\right)^{\frac{3}{2}}}\\- \frac{w z}{\left(w^{2} + x^{2} + y^{2} + z^{2}\right)^{\frac{3}{2}}}\end{matrix}\right] $$

$$ \frac{\partial q}{\partial x} = \left[\begin{matrix}- \frac{w x}{\left(w^{2} + x^{2} + y^{2} + z^{2}\right)^{\frac{3}{2}}}\\- \frac{x^{2}}{\left(w^{2} + x^{2} + y^{2} + z^{2}\right)^{\frac{3}{2}}} + \frac{1}{\sqrt{w^{2} + x^{2} + y^{2} + z^{2}}}\\- \frac{x y}{\left(w^{2} + x^{2} + y^{2} + z^{2}\right)^{\frac{3}{2}}}\\- \frac{x z}{\left(w^{2} + x^{2} + y^{2} + z^{2}\right)^{\frac{3}{2}}}\end{matrix}\right] $$

$$ \frac{\partial q}{\partial y} =\left[\begin{matrix}- \frac{w y}{\left(w^{2} + x^{2} + y^{2} + z^{2}\right)^{\frac{3}{2}}}\\- \frac{x y}{\left(w^{2} + x^{2} + y^{2} + z^{2}\right)^{\frac{3}{2}}}\\- \frac{y^{2}}{\left(w^{2} + x^{2} + y^{2} + z^{2}\right)^{\frac{3}{2}}} + \frac{1}{\sqrt{w^{2} + x^{2} + y^{2} + z^{2}}}\\- \frac{y z}{\left(w^{2} + x^{2} + y^{2} + z^{2}\right)^{\frac{3}{2}}}\end{matrix}\right] $$ 

$$ \frac{\partial q}{\partial z} = \left[\begin{matrix}- \frac{w z}{\left(w^{2} + x^{2} + y^{2} + z^{2}\right)^{\frac{3}{2}}}\\- \frac{x z}{\left(w^{2} + x^{2} + y^{2} + z^{2}\right)^{\frac{3}{2}}}\\- \frac{y z}{\left(w^{2} + x^{2} + y^{2} + z^{2}\right)^{\frac{3}{2}}}\\- \frac{z^{2}}{\left(w^{2} + x^{2} + y^{2} + z^{2}\right)^{\frac{3}{2}}} + \frac{1}{\sqrt{w^{2} + x^{2} + y^{2} + z^{2}}}\end{matrix}\right] $$

In [56]:
import torch
from torch.autograd import Function, gradcheck

class QuaternionNormalization(Function):
    @staticmethod
    def forward(ctx, q):
        q_norm = q / torch.norm(q, dim=1, keepdim=True)
        ctx.save_for_backward(q)
        return q_norm

    @staticmethod
    def backward(ctx, grad_q_norm):
        q = ctx.saved_tensors[0]
        w = q[:, 0]
        x = q[:, 1]
        y = q[:, 2]
        z = q[:, 3]
        
        norm_sq = w * w + x * x + y * y + z * z
        grad_qw = (-1 * w * w / norm_sq**1.5 + 1/norm_sq**0.5) * grad_q_norm[:, 0] - w * x / norm_sq**1.5 * grad_q_norm[:, 1] - w * y / norm_sq**1.5 * grad_q_norm[:, 2] - w * z / norm_sq**1.5 * grad_q_norm[:, 3]
        grad_qx = -w * x / norm_sq**1.5 * grad_q_norm[:, 0] + (-1 * x * x / norm_sq**1.5 + 1/norm_sq**0.5) * grad_q_norm[:, 1] - x * y / norm_sq**1.5 * grad_q_norm[:, 2] - x * z / norm_sq**1.5 * grad_q_norm[:, 3]
        grad_qy = -w * y / norm_sq**1.5 * grad_q_norm[:, 0] - x * y / norm_sq**1.5 * grad_q_norm[:, 1] + (-1 * y * y / norm_sq**1.5 + 1/norm_sq**0.5) * grad_q_norm[:, 2] - y * z / norm_sq**1.5 * grad_q_norm[:, 3]
        grad_qz = -w * z / norm_sq**1.5 * grad_q_norm[:, 0] - x * z / norm_sq**1.5 * grad_q_norm[:, 1] - y * z / norm_sq**1.5 * grad_q_norm[:, 2] + (-1 * z * z / norm_sq**1.5 + 1/norm_sq**0.5) * grad_q_norm[:, 3]
        grad_q = torch.stack([grad_qw, grad_qx, grad_qy, grad_qz], dim=1)

        return grad_q
        

q = torch.rand(2, 4, dtype=torch.float64, requires_grad=True)

test = gradcheck(QuaternionNormalization.apply, (q))
print(test)

True


In [21]:
import torch
from torch.autograd import Function, gradcheck

class ComputeSigmaWorld(Function):
    @staticmethod
    def forward(ctx, q, scale):
        S = torch.diag_embed(torch.exp(scale))
        norm_q = torch.norm(q, dim=1, keepdim=True)
        q_norm = q / norm_q
        R = [
            1 - 2 * q_norm[:, 2] ** 2 - 2 * q_norm[:, 3] ** 2,
            2 * q_norm[:, 1] * q_norm[:, 2] - 2 * q_norm[:, 0] * q_norm[:, 3],
            2 * q_norm[:, 3] * q_norm[:, 1] + 2 * q_norm[:, 0] * q_norm[:, 2],
            2 * q_norm[:, 1] * q_norm[:, 2] + 2 * q_norm[:, 0] * q_norm[:, 3],
            1 - 2 * q_norm[:, 1] ** 2 - 2 * q_norm[:, 3] ** 2,
            2 * q_norm[:, 2] * q_norm[:, 3] - 2 * q_norm[:, 0] * q_norm[:, 1],
            2 * q_norm[:, 3] * q_norm[:, 1] - 2 * q_norm[:, 0] * q_norm[:, 2],
            2 * q_norm[:, 2] * q_norm[:, 3] + 2 * q_norm[:, 0] * q_norm[:, 1],
            1 - 2 * q_norm[:, 1] ** 2 - 2 * q_norm[:, 2] ** 2,
        ]
        R = torch.stack(R, dim=1).reshape(-1, 3, 3)

        RS = torch.bmm(R, S)
        RS_t = RS.permute(0, 2, 1)

        RSSR = torch.bmm(RS, RS_t)
        ctx.save_for_backward(RS, R, S, scale, q, q_norm)
        return RSSR
    
    @staticmethod
    def backward(ctx, grad_RSSR):
        # compute double matmul gradient        
        RS, R, S, scale, q, q_norm = ctx.saved_tensors
        grad_RS = torch.bmm(grad_RSSR, RS)
        
        RS_t = RS.permute(0, 2, 1)
        grad_SR = RS_t @ grad_RSSR

        grad_R = grad_RS @ S.permute(0, 2, 1) + (S @ grad_SR).permute(0, 2, 1)
        grad_S = R.permute(0, 2, 1) @ grad_RS + (grad_SR @ R).permute(0, 2, 1)

        # compute quaternion gradient
        w = q_norm[:, 0]
        x = q_norm[:, 1]
        y = q_norm[:, 2]
        z = q_norm[:, 3]
        grad_qw_norm = -2 * z *grad_R[:,0 , 1] + 2 * y *grad_R[:,0, 2] + 2 * z *grad_R[:,1, 0] - \
            2 * x *grad_R[:,1, 2] - 2 * y *grad_R[:,2, 0] + 2 * x *grad_R[:,2, 1]
        grad_qx_norm = 2 * y *grad_R[:,0, 1] + 2 * z *grad_R[:,0, 2] + 2 * y *grad_R[:,1, 0] - \
            4 * x *grad_R[:,1, 1] - 2 * w *grad_R[:,1, 2] + 2 * z *grad_R[:,2, 0] + 2 * w *grad_R[:,2, 1] - 4 * x *grad_R[:,2, 2]
        grad_qy_norm = -4 * y *grad_R[:,0, 0] + 2 * x *grad_R[:,0, 1] + 2 * w *grad_R[:,0, 2] + \
            2 * x *grad_R[:,1, 0] + 2 * z *grad_R[:,1, 2] - 2 * w *grad_R[:,2, 0] + 2 * z *grad_R[:,2, 1] - 4 * y *grad_R[:,2, 2]
        grad_qz_norm = -4 * z *grad_R[:,0, 0] - 2 * w *grad_R[:,0, 1] + 2 * x *grad_R[:,0, 2] + \
            2 * w *grad_R[:,1, 0] - 4 * z *grad_R[:,1, 1] + 2 * y *grad_R[:,1, 2] + 2 * x *grad_R[:,2, 0] + 2 * y *grad_R[:,2, 1]
        grad_q_norm = torch.stack([grad_qw_norm, grad_qx_norm, grad_qy_norm, grad_qz_norm], dim=1)

        # compute gradient for unnormalized quaternion
        w = q[:, 0]
        x = q[:, 1]
        y = q[:, 2]
        z = q[:, 3]
        norm_sq = w * w + x * x + y * y + z * z
        grad_qw = (-1 * w * w / norm_sq**1.5 + 1/norm_sq**0.5) * grad_q_norm[:, 0] - w * x / norm_sq**1.5 * grad_q_norm[:, 1] - \
            w * y / norm_sq**1.5 * grad_q_norm[:, 2] - w * z / norm_sq**1.5 * grad_q_norm[:, 3]
        grad_qx = -w * x / norm_sq**1.5 * grad_q_norm[:, 0] + (-1 * x * x / norm_sq**1.5 + 1/norm_sq**0.5) * grad_q_norm[:, 1] - \
            x * y / norm_sq**1.5 * grad_q_norm[:, 2] - x * z / norm_sq**1.5 * grad_q_norm[:, 3]
        grad_qy = -w * y / norm_sq**1.5 * grad_q_norm[:, 0] - x * y / norm_sq**1.5 * grad_q_norm[:, 1] + (-1 * y * y / norm_sq**1.5 + \
            1/norm_sq**0.5) * grad_q_norm[:, 2] - y * z / norm_sq**1.5 * grad_q_norm[:, 3]
        grad_qz = -w * z / norm_sq**1.5 * grad_q_norm[:, 0] - x * z / norm_sq**1.5 * grad_q_norm[:, 1] - y * z / norm_sq**1.5 * grad_q_norm[:, 2] + \
            (-1 * z * z / norm_sq**1.5 + 1/norm_sq**0.5) * grad_q_norm[:, 3]
        grad_q = torch.stack([grad_qw, grad_qx, grad_qy, grad_qz], dim=1)

        grad_scale_no_activation = grad_S.diagonal(dim1=1, dim2=2)
        grad_scale = grad_scale_no_activation * torch.exp(scale)

        return grad_q, grad_scale
    
N = 2
q = torch.rand(N, 4, dtype=torch.float64, requires_grad=True)

s = torch.rand(N, 3, dtype=torch.float64, requires_grad=True)
test = gradcheck(ComputeSigmaWorld.apply, (q, s))
print(test)

True
